LAb 15

rollno 20SW022

sec 2

In [3]:
!pip install scikit-surprise

# Import libraries
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163332 sha256=94324f1346996025ebe3cf22eeeb72741de6f1f46989d0e60f10c4e8e6636d3c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
df = pd.read_csv('/content/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [5]:
df.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [6]:
df.describe()

,user_id,book_id,rating
count,5.976479e+06,5.976479e+06,5.976479e+06
mean,2.622446e+04,2.006477e+03,3.919866e+00
std,1.541323e+04,2.468499e+03,9.910868e-01
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.281300e+04,1.980000e+02,3.000000e+00
50%,2.593800e+04,8.850000e+02,4.000000e+00
75%,3.950900e+04,2.973000e+03,5.000000e+00
max,5.342400e+04,1.000000e+04,5.000000e+00


In [7]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)


In [8]:
# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [9]:
# Build the recommendation model (SVD algorithm)
model = SVD(n_factors=100, n_epochs=20, verbose=True)
model.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [10]:

# Make predictions on the test set
predictions = model.test(testset)

In [11]:
# Evaluate the model
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

RMSE: 0.8313
MAE:  0.6414
RMSE: 0.8313142997205492
MAE: 0.6413848575003273


In [12]:
# Create a function to recommend books for a user
def get_top_n(predictions, n=10):
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [13]:
# Get recommendations for a user (change 'user_id' to your target user)
user_id = 1
top_n = get_top_n(predictions, n=10)
recommended_books = top_n[user_id]
print(f"Top 10 recommended books for User {user_id}:")
for book_id, est_rating in recommended_books:
    print(f"Book ID: {book_id}, Estimated Rating: {est_rating}")

Top 10 recommended books for User 1:
Book ID: 2535, Estimated Rating: 4.205106856465043
Book ID: 2063, Estimated Rating: 3.973035113048409
Book ID: 102, Estimated Rating: 3.9596388094079673
Book ID: 72, Estimated Rating: 3.8618009731410785
Book ID: 231, Estimated Rating: 3.8320021850467993
Book ID: 1041, Estimated Rating: 3.8308180618373906
Book ID: 421, Estimated Rating: 3.815595540387359
Book ID: 40, Estimated Rating: 3.7941665357291674
Book ID: 67, Estimated Rating: 3.7727932912941817
Book ID: 11, Estimated Rating: 3.7280627758715683


In [14]:
# Get recommendations for a user (change 'user_id' to your target user)
user_id = 2
top_n = get_top_n(predictions, n=10)
recommended_books = top_n[user_id]
print(f"Top 10 recommended books for User {user_id}:")
for book_id, est_rating in recommended_books:
    print(f"Book ID: {book_id}, Estimated Rating: {est_rating}")

Top 10 recommended books for User 2:
Book ID: 3753, Estimated Rating: 5
Book ID: 18, Estimated Rating: 4.955001340230349
Book ID: 360, Estimated Rating: 4.498845565501635
Book ID: 2318, Estimated Rating: 4.4600389917320244
Book ID: 82, Estimated Rating: 4.296176218873626
Book ID: 15, Estimated Rating: 4.22624696397249
Book ID: 790, Estimated Rating: 4.186975452449065
Book ID: 1446, Estimated Rating: 4.155265281030815
Book ID: 315, Estimated Rating: 3.5926826661335203
